In [ ]:
### SCRIPT PER CLONARE REPO GITHUB CON GIT LFS ###

import os

# Configura percorso
base_path = '/content/Progetti_IA'
repo_name = 'SegmentazioneNoduliCerebrali'
repo_url = 'https://github.com/s-villanova/SegmentazioneNoduliCerebrali.git'

# Crea la cartella base se non esiste
os.makedirs(base_path, exist_ok=True)
repo_path = os.path.join(base_path, repo_name)

# Installa Git LFS
!apt-get install git-lfs -y
!git lfs install

if os.path.exists(repo_path):
    print(f"📂 La cartella '{repo_name}' esiste già. Eseguo git pull...")
    %cd {repo_path}
    !git stash --include-untracked --quiet
    !git pull
    !git lfs pull
else:
    print(f"⬇️ Clono il repository '{repo_name}'...")
    %cd {base_path}
    !git clone {repo_url}
    %cd {repo_name}
    !git lfs pull

print("✅ Repository pronto!")

In [ ]:
### SCRIPT PER SALVARE BACKUP MODELLO E DATASET SU DRIVE ###

from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/SegmentazioneNoduliCerebrali /content/drive/MyDrive/Backup_SegmentazioneNoduliCerebrali/


In [ ]:
### SCRIPT PER INSTALLARE DETECTRON PYTORCH E TORCHVISION ###

# PyTorch + torchvision compatibili col runtime CUDA
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113

# PyTorch + torchvision compatibili col runtime CPU
#!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
%cd /content/drive/MyDrive/Progetti_IA/SegmentazioneNoduliCerebrali/
# 2. Installa dipendenze di compilazione
!pip install pyyaml==6.0 cython 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

%cd detectron2
!pip install -e .



In [ ]:
### SCRIPT PER ESTRAZIONE DI EVENTUALI FILE ZIP ###

import zipfile
import os

# Percorso al file ZIP su Drive
zip_path = '/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/test/images.zip'

# Cartella di destinazione
extract_path = '/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/test/images/'

# Crea la cartella se non esiste
os.makedirs(extract_path, exist_ok=True)

# Estrazione
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("File scompattato correttamente in:", extract_path)


In [ ]:
### SCRIPT PER IMPORTARE ISTANZE COCO ###


# 1) Importa
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances

# 2) Registra le partizioni
register_coco_instances(
    "brain_train",       # nome del dataset in Detectron2
    {},                  # metadati (lascia vuoto se non ne hai di custom)
    "/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/train/annotation.coco.json",
    "/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/train/images"
)

register_coco_instances(
    "brain_test",
    {},
    "/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/test/annotation.coco.json",
    "/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/test/images"
)

# 3) Verifica
for split in ["brain_train", "brain_test"]:
    data = DatasetCatalog.get(split)
    meta = MetadataCatalog.get(split)
    print(f"{split}: {len(data)} immagini, categorie = {meta.thing_classes if hasattr(meta, 'thing_classes') else 'non definito'}")


In [ ]:
### SCRIPT PER TRAINING MODELLO ###


import copy
import os
import warnings

import torch
import torch.nn.functional as F
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, HookBase
from detectron2.data import build_detection_train_loader, detection_utils as utils
from detectron2.data import transforms as T
from detectron2.modeling import GeneralizedRCNN, META_ARCH_REGISTRY

# --- 1) Dice loss ---
def dice_loss(pred_mask, target_mask, smooth=1e-6):
    pred_mask = pred_mask.sigmoid()
    inter = (pred_mask * target_mask).sum(dim=[1,2])
    union = pred_mask.sum(dim=[1,2]) + target_mask.sum(dim=[1,2])
    return (1 - (2*inter+smooth)/(union+smooth)).mean()

# --- 2) Hook per aggiungere il Dice loss dopo la mask_loss standard ---
class DiceLossHook(HookBase):
    def after_step(self):
        """
        Viene chiamato *dopo* che trainer.model.losses è stato calcolato.
        Qui recuperiamo:
          - il dizionario self.trainer.storage.latest(), contenente 'loss_mask'
          - i tensor self.trainer.model.roi_heads._last_mask_logits e _last_gt_masks
        e sommiamo il dice loss.
        """
        metrics = self.trainer.storage._latest  # dict di losses correnti
        # recupera i logits e le gt masks salvati in training
        mh = self.trainer.model.roi_heads
        if hasattr(mh, "_last_mask_logits") and hasattr(mh, "_last_gt_masks"):
            logits = mh._last_mask_logits
            gt_masks = mh._last_gt_masks
            if logits is not None and gt_masks is not None:
                d = dice_loss(logits, gt_masks)
                # somma in-place
                metrics["loss_mask"] += d.item()
                # e segnala al backprop
                self.trainer.storage._latest["loss_mask"] = metrics["loss_mask"]
                # aggiungi il dice al dict per il backward
                self.trainer.storage._latest["d2_dice_loss"] = d.item()
                # infine, aggiungi al grad
                self.trainer.model._losses["loss_mask"] = (
                    self.trainer.model._losses["loss_mask"] + d
                )



# Ambient and warning setup
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
warnings.filterwarnings("ignore", category=FutureWarning)

# Configuration
cfg = get_cfg()
cfg.merge_from_file("/content/Progetti_IA/SegmentazioneNoduliCerebrali/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.META_ARCHITECTURE = "GeneralizedRCNN"
cfg.MODEL.MASK_ON = True

cfg.DATASETS.TRAIN = ("brain_train",)
cfg.DATASETS.TEST = ("brain_test",)

cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 14
cfg.SOLVER.BASE_LR = 0.004
cfg.INPUT.MASK_FORMAT = "bitmask"

cfg.SOLVER.AMP.ENABLED = True
cfg.MODEL.DEVICE = "cuda" # usa "cpu" se non hai GPU

cfg.SOLVER.MAX_ITER = 8000
cfg.SOLVER.GAMMA = 0.6
cfg.SOLVER.STEPS = [3000, 6000]
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.WARMUP_METHOD = "linear"
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 1000
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.DETECTIONS_PER_IMAGE = 5
cfg.INPUT.MIN_SIZE_TRAIN = (512,)
cfg.INPUT.MAX_SIZE_TRAIN = 512
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
# Output
cfg.OUTPUT_DIR = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented"
#cfg.MODEL.WEIGHTS = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/model_final.pth"

# === 3) Custom mapper per COCO RLE ===
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    # 1) Definisci le stesse augmentations
    augmentation_list = [
        T.Resize((512, 512)),
        T.RandomFlip(prob=0.5),
        T.RandomRotation(angle=[-15, 15]),
        T.RandomCrop(crop_type="relative_range", crop_size=[0.8, 0.8]),
    ]

    # 2) Applica le augmentations all’immagine
    aug_input = T.AugInput(image)
    transforms = T.AugmentationList(augmentation_list)(aug_input)
    image = aug_input.image
    new_h, new_w = image.shape[:2]

    # 3) **RICAMPIONA** le annotation con le stesse transforms!
    annos = []
    for ann in dataset_dict["annotations"]:
        ann_trans = utils.transform_instance_annotations(
            ann, transforms, (new_h, new_w)
        )
        annos.append(ann_trans)

    # 4) Crea finalmente le Instances “allineate”
    instances = utils.annotations_to_instances(
        annos, (new_h, new_w), mask_format="bitmask"
    )

    # 5) Ritorna tutto nel formato detectron2
    dataset_dict["image"]     = torch.as_tensor(image.transpose(2,0,1), dtype=torch.float32)
    dataset_dict["instances"] = instances
    return dataset_dict

# === 4) Trainer custom con mapper ===
class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

# === 5) Training ===
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
### SCRIPT DEBUG PER VALIDARE MAPER E AUGMENTATION USATI IN TRAINING ###

import numpy as np
import random
import copy
import matplotlib.pyplot as plt
import torch
from detectron2.data import DatasetCatalog, detection_utils as utils
from detectron2.data import transforms as T

dataset_dicts = DatasetCatalog.get("brain_train")

def debug_mapper(dataset_dict):
    d = copy.deepcopy(dataset_dict)
    img = utils.read_image(d["file_name"], format="BGR")  # uint8 0–255
    h0, w0 = img.shape[:2]

    aug_list = [
        T.Resize((512, 512)),
        T.RandomFlip(prob=0.5),
        T.RandomRotation(angle=[-15, 15]),
        T.RandomCrop(crop_type="relative_range", crop_size=[0.8, 0.8]),
    ]
    aug_input = T.AugInput(img)
    transforms = T.AugmentationList(aug_list)(aug_input)
    img_trans = aug_input.image   # ancora uint8 0–255
    h, w = img_trans.shape[:2]

    annos = []
    for ann in d["annotations"]:
        annos.append(utils.transform_instance_annotations(
            ann, transforms, (h, w)
        ))
    instances = utils.annotations_to_instances(
        annos, (h, w), mask_format="bitmask"
    )

    # Restituisco l'immagine raw e le instances
    return img_trans, instances

# Visualizzo 4 campioni
fig, axes = plt.subplots(2, 2, figsize=(10,10))
axes = axes.flatten()
for ax in axes:
    sample = random.choice(dataset_dicts)
    img, inst = debug_mapper(sample)

    # BGR -> RGB
    img_rgb = img[:, :, ::-1]

    ax.imshow(img_rgb)
    # contorni delle maschere
    masks = inst.gt_masks.tensor.cpu().numpy()  # [N,H,W]
    for m in masks:
        ax.contour(m, levels=[0.5], colors='r', linewidths=1)

    ax.set_title(f"Image ID {sample['image_id']}")
    ax.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
### SCRIPT PER EVALUATION CON COCO EVALUATOR ###

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# 1) Configurazione come prima
cfg = get_cfg()
cfg.merge_from_file("/content/Progetti_IA/SegmentazioneNoduliCerebrali/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TEST = ("brain_test",)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.MODEL.WEIGHTS = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/model_0007999.pth"
cfg.OUTPUT_DIR = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented_eval"

cfg.MODEL.DEVICE = "cuda" # usa "cpu" se non hai GPU

# 2) DefaultPredictor costruisce internamente il modello e il preprocess
predictor = DefaultPredictor(cfg)

# 3) COCOEvaluator sullo split brain_test
evaluator = COCOEvaluator("brain_test", cfg, False, output_dir=cfg.OUTPUT_DIR)

# 4) DataLoader di test
test_loader = build_detection_test_loader(cfg, "brain_test")

# 5) Inference + metriche
res = inference_on_dataset(predictor.model, test_loader, evaluator)
print(res)


In [ ]:
### SCRIPT PER VISUALIZZARE EVENTI TENSORBOARD ###

%load_ext tensorboard
%tensorboard --logdir /content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/


In [ ]:
### SCRIPT PER ESEGUIRE INFERENZA CON IMMAGINE CARICATA DAL PROPRIO PC ###

import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from google.colab.patches import cv2_imshow
from google.colab import files

# ─────── 1) Configura il modello ───────
cfg = get_cfg()

cfg.merge_from_file("/content/Progetti_IA/SegmentazioneNoduliCerebrali/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TEST = ("brain_test",)
cfg.DATASETS.TRAIN = ("brain_train",)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.DEVICE = "cuda"  # usa "cpu" se non hai GPU

cfg.MODEL.WEIGHTS = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/model_0007999.pth"

predictor = DefaultPredictor(cfg)

# ─────── 2) Carica immagine ───────
print("▶️ Carica un’immagine dal tuo PC:")
uploaded = files.upload()
file_name = next(iter(uploaded.keys()))
image = cv2.imread(file_name)

# ─────── 3) Inferenza ───────
outputs = predictor(image)

# ─────── 4) Visualizza risultati ───────
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])

instances = outputs["instances"].to("cpu")

v = Visualizer(image[:, :, ::-1], metadata=metadata, scale=1.2)
out = v.draw_instance_predictions(instances)

cv2_imshow(out.get_image()[:, :, ::-1])


In [ ]:
### SCRIPT GRADIO PER TESTING MODELLO ###


import os
import cv2
import numpy as np
import gradio as gr
import torch
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog, Metadata
from detectron2.structures import Instances, BitMasks, Boxes
from detectron2.utils.visualizer import Visualizer, ColorMode
from pycocotools import mask as maskUtils
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    jaccard_score, balanced_accuracy_score,
    confusion_matrix
)
from scipy.spatial.distance import directed_hausdorff
import traceback

from detectron2.utils.colormap import random_color



def no_jitter_color(color, brightness_factor=1.0):
    # Ignora qualsiasi modifica e restituisce sempre il colore originale
    return color

# Sovrascrivi la funzione privata _jitter_color usata da Visualizer
Visualizer._jitter_color = staticmethod(no_jitter_color)


# ─────── CONFIG ───────
cfg = get_cfg()
cfg.merge_from_file(
    "/content/Progetti_IA/SegmentazioneNoduliCerebrali/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
)
cfg.DATASETS.TEST = ("brain_test",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.WEIGHTS = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/model_0007999.pth"

predictor = DefaultPredictor(cfg)
orig_metadata = MetadataCatalog.get("brain_test")

PRED_COLOR = (255, 255, 0)
GT_COLOR   = (76, 145, 255)

def fixed_random_color(class_index=0, *, rgb=False):
    # Sempre lo stesso colore, per esempio GT_COLOR normalizzato a 0-1 se rgb=True
    color = np.array(GT_COLOR, dtype=np.float32) / 255.0 if rgb else GT_COLOR
    return tuple(color)

# Patch
import detectron2.utils.colormap as colormap_module
colormap_module.random_color = fixed_random_color

dataset_dicts = DatasetCatalog.get("brain_test")
immagini = [os.path.basename(d["file_name"]) for d in dataset_dicts if d.get("annotations")]
NUM_CLASSES = cfg.MODEL.ROI_HEADS.NUM_CLASSES

# ─────── UTILS ───────
def segm_to_mask(segm, h, w):
    if isinstance(segm, list):
        rles = maskUtils.frPyObjects(segm, h, w)
        rle  = maskUtils.merge(rles)
    elif isinstance(segm, dict) and isinstance(segm.get("counts"), list):
        rle = maskUtils.frPyObjects(segm, h, w)
    else:
        rle = segm
    return maskUtils.decode(rle).astype(np.uint8)

def hausdorff(u, v):
    u_pts = np.argwhere(u)
    v_pts = np.argwhere(v)
    if u_pts.size == 0 or v_pts.size == 0:
        return 0.0
    return max(
        directed_hausdorff(u_pts, v_pts)[0],
        directed_hausdorff(v_pts, u_pts)[0]
    )

def crea_overlay(gt_all, pred_all, canvas_size=512, max_crop=300):
    gt_mask = gt_all.sum(axis=0) > 0
    pr_mask = pred_all.sum(axis=0) > 0
    both    = gt_mask & pr_mask
    only_gt = gt_mask & ~pr_mask
    only_pr = pr_mask & ~gt_mask

    H, W = gt_mask.shape
    overlay = np.ones((H, W, 3), dtype=np.uint8) * 240
    overlay[only_gt] = GT_COLOR
    overlay[only_pr] = PRED_COLOR
    overlay[both]   = (28, 255, 28)



    coords = np.argwhere(gt_mask | pr_mask)
    if coords.size == 0:
        return np.ones((canvas_size, canvas_size, 3), dtype=np.uint8) * 240
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    crop = overlay[y0:y1, x0:x1]
    crop = cv2.copyMakeBorder(crop, 1,1,1,1, cv2.BORDER_CONSTANT, value=(240,240,240))
    h, w = crop.shape[:2]
    scale = max_crop / max(h, w)
    crop = cv2.resize(crop, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_NEAREST)

    canvas = np.ones((canvas_size, canvas_size, 3), dtype=np.uint8) * 240
    ys = (canvas_size - crop.shape[0]) // 2
    xs = (canvas_size - crop.shape[1]) // 2
    canvas[ys:ys+crop.shape[0], xs:xs+crop.shape[1]] = crop
    return canvas
def build_gt_instances(gt_item, H, W):
    masks = []
    classes = []
    boxes = []
    for ann in gt_item.get("annotations", []):
        m = segm_to_mask(ann["segmentation"], H, W).astype(bool)
        masks.append(m)
        classes.append(ann["category_id"])
        # Convert [x, y, w, h] to [x1, y1, x2, y2]
        x, y, w, h = ann["bbox"]
        boxes.append([x, y, x + w, y + h])
    inst = Instances((H, W))
    inst.pred_masks = BitMasks(np.array(masks))
    inst.pred_classes = np.array(classes)
    inst.pred_boxes = Boxes(torch.tensor(boxes))
    return inst

# ─────── MAIN FUNCTION ───────
def visualizza_con_metriche(nome_file):
    try:
        img_path = os.path.join(
            "/content/Progetti_IA/SegmentazioneNoduliCerebrali/BRISC2025/segmentation_task/test/images",
            nome_file
        )
        if not os.path.isfile(img_path):
            return None, None, None, None, f"❌ File non trovato: {img_path}"
        img = cv2.imread(img_path)
        img_rgb = img[:, :, ::-1]
        H, W = img.shape[:2]

        out = predictor(img)
        inst = out["instances"].to("cpu")

        gt_item = next((d for d in dataset_dicts if os.path.basename(d["file_name"]) == nome_file), None)
        if gt_item is None:
            return img_rgb, None, None, None, "❌ GT non trovata!"

        gt_all   = np.zeros((NUM_CLASSES, H, W), dtype=bool)
        pred_all = np.zeros((NUM_CLASSES, H, W), dtype=bool)
        for ann in gt_item.get("annotations", []):
            m = segm_to_mask(ann["segmentation"], H, W).astype(bool)
            gt_all[ann["category_id"]] |= m
        for i, m in enumerate(inst.pred_masks if inst.has("pred_masks") else []):
            cls = inst.pred_classes[i]
            pred_all[cls] |= m.numpy()

        report = ""
        for cls in range(NUM_CLASSES):
            yt = gt_all[cls].ravel(); yp = pred_all[cls].ravel()
            if not (yt.any() or yp.any()): continue
            iou  = jaccard_score(yt, yp, zero_division=0)
            dice = f1_score(yt, yp, zero_division=0)
            prec = precision_score(yt, yp, zero_division=0)
            rec  = recall_score(yt, yp, zero_division=0)
            tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0,1]).ravel()
            spec   = tn / (tn + fp + 1e-6)
            balacc = balanced_accuracy_score(yt, yp)
            hd     = hausdorff(gt_all[cls], pred_all[cls])
            report += (f"Classe {cls}: IoU {iou:.3f}, DICE {dice:.3f}, Prec {prec:.3f}, "
                       f"Rec {rec:.3f}, Spec {spec:.3f}, BalAcc {balacc:.3f}, "
                       f"Hausdorff {hd:.1f}px\n")

        meta_pred = Metadata()
        meta_pred.thing_classes = orig_metadata.thing_classes
        meta_pred.thing_colors  = [PRED_COLOR] * NUM_CLASSES
        vp = Visualizer(img_rgb, meta_pred, scale=1.0, instance_mode=ColorMode.SEGMENTATION)
        vp._default_mask_alpha = 1.0
        vis_pred = vp.draw_instance_predictions(inst).get_image()

        meta_gt = Metadata()
        meta_gt.thing_classes = orig_metadata.thing_classes
        meta_gt.thing_colors  = [GT_COLOR] * NUM_CLASSES
        gt_inst = build_gt_instances(gt_item, H, W)
        vg = Visualizer(img_rgb, meta_gt, scale=1.0, instance_mode=ColorMode.SEGMENTATION)
        vg._default_mask_alpha = 1.0
        vis_gt = vg.draw_instance_predictions(gt_inst).get_image()

        overlay = crea_overlay(gt_all.astype(np.uint8), pred_all.astype(np.uint8))
        return img_rgb, vis_pred, vis_gt, overlay, report
    except Exception as e:
        print(traceback.format_exc())
        return None, None, None, None, f"❌ Errore interno:\n{e}"

if __name__ == "__main__":
    gr.Interface(
        fn=visualizza_con_metriche,
        inputs=gr.Dropdown(choices=immagini, label="Seleziona immagine"),
        outputs=[
            gr.Image(label="Originale"),
            gr.Image(label="Predizione"),
            gr.Image(label="Ground Truth"),
            gr.Image(label="Overlay: Giallo=Predetto, Blu=GT, Verde=Comune"),
            gr.Textbox(label="Metriche", lines=8)
        ],
        title="Segmentation QA con metriche su sfondo bianco",
        allow_flagging="never"
    ).launch(debug=True)


In [ ]:
### SCRIPT PER INFERENZA E GENERAZIONE REPORT CSV ###

import os
import csv
import cv2
import numpy as np
from scipy.spatial.distance import directed_hausdorff

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from pycocotools import mask as maskUtils

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    jaccard_score,
    balanced_accuracy_score,
    confusion_matrix,
)

# ────────── CONFIGURAZIONE ──────────
cfg = get_cfg()
cfg.merge_from_file("/content/Progetti_IA/SegmentazioneNoduliCerebrali/detectron2/configs/"
                    "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TEST = ("brain_test",)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.INPUT.MIN_SIZE_TEST = 512
cfg.INPUT.MAX_SIZE_TEST = 512
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # soglia alta
cfg.MODEL.DEVICE = "cuda" # usa "cpu" se non hai GPU
cfg.MODEL.WEIGHTS = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/model/augmented/model_0007999.pth"

predictor = DefaultPredictor(cfg)
metadata  = MetadataCatalog.get("brain_test")
dataset_dicts = DatasetCatalog.get("brain_test")
thing_classes = metadata.thing_classes

# ────────── UTILITY ──────────
def segm_to_mask(segm, h, w):
    """Da COCO RLE/poligono a maschera binaria HxW."""
    if isinstance(segm, list):
        rles = maskUtils.frPyObjects(segm, h, w)
        rle  = maskUtils.merge(rles)
    elif isinstance(segm.get("counts"), list):
        rle = maskUtils.frPyObjects(segm, h, w)
    else:
        rle = segm
    return maskUtils.decode(rle).astype(np.uint8)

def hausdorff(u, v):
    uc = np.argwhere(u)
    vc = np.argwhere(v)
    if len(uc)==0 or len(vc)==0:
        return 0.0
    return max(
        directed_hausdorff(uc, vc)[0],
        directed_hausdorff(vc, uc)[0]
    )

# ────────── COLLECT STATS ──────────
rows = []
# per classe GT: (correct, wrong)
agg = {cls: {"correct":0, "wrong":0} for cls in thing_classes}

n_found = 0
n_not_found = 0

for idx, item in enumerate(dataset_dicts, 1):
    fname = os.path.basename(item["file_name"])
    img_bgr = cv2.imread(item["file_name"])
    h, w = img_bgr.shape[:2]

    outputs = predictor(img_bgr)
    inst = outputs["instances"].to("cpu")

    # trova la predizione con conf più alta, se c'è
    if len(inst) == 0:
        pred_cls = ""
        pred_mask = np.zeros((h,w), np.uint8)
        n_not_found += 1
    else:
        scores = inst.scores.numpy()
        best = scores.argmax()
        pred_cls = int(inst.pred_classes[best])
        pred_mask = inst.pred_masks[best].numpy().astype(np.uint8)
        n_found += 1

    # ground-truth unico per immagine (assumiamo 1 sola annotazione per immagine)
    gt_ann = item["annotations"][0]
    gt_cls = gt_ann["category_id"]
    gt_mask = segm_to_mask(gt_ann["segmentation"], h, w)

    # match classe?
    match = 1 if (pred_cls == gt_cls) else 0
    if pred_cls == "":
        # nessuna predizione
        metrics = dict(iou=0,dice=0,precision=0,recall=0,
                       specificity=0,balanced_acc=0,hausdorff_px=0)
    else:
        # calcola metriche pixel-wise
        yt = gt_mask.flatten()
        yp = pred_mask.flatten()
        iou    = jaccard_score(yt, yp, zero_division=0)
        dice   = f1_score(yt, yp, zero_division=0)
        prec   = precision_score(yt, yp, zero_division=0)
        rec    = recall_score(yt, yp, zero_division=0)
        balacc = balanced_accuracy_score(yt, yp)
        tn, fp, fn, tp = confusion_matrix(yt, yp, labels=[0,1]).ravel()
        spec   = tn / (tn + fp + 1e-6)
        hd     = hausdorff(gt_mask>0, pred_mask>0)
        metrics = dict(iou=iou, dice=dice, precision=prec,
                       recall=rec, specificity=spec,
                       balanced_acc=balacc, hausdorff_px=hd)

    # aggiorna aggregati per classe GT
    cls_name = thing_classes[gt_cls]
    if pred_cls == gt_cls:
        agg[cls_name]["correct"] += 1
    else:
        agg[cls_name]["wrong"]   += 1

    rows.append([
        fname,
        cls_name,
        thing_classes[pred_cls] if pred_cls!="" else "",
        match,
        f"{metrics['iou']:.4f}",
        f"{metrics['dice']:.4f}",
        f"{metrics['precision']:.4f}",
        f"{metrics['recall']:.4f}",
        f"{metrics['specificity']:.4f}",
        f"{metrics['balanced_acc']:.4f}",
        f"{metrics['hausdorff_px']:.2f}",
    ])

    print(f"[{idx}/{len(dataset_dicts)}] {fname} → GT={cls_name}  PRED="
          f"{metrics['iou']:.3f}, match={match}")

# calcola global means su tumori trovati
import pandas as pd
df = pd.DataFrame(rows, columns=[
    "image","gt_class","pred_class","match",
    "iou","dice","precision","recall",
    "specificity","balanced_acc","hausdorff_px"
])
df.to_csv("/content/Progetti_IA/SegmentazioneNoduliCerebrali/report/metrics_per_image.csv", index=False)

# prepariamo gli aggregati per classe
agg_rows = []
for cls, v in agg.items():
    agg_rows.append([cls, v["correct"], v["wrong"]])

# medie globali (solo match o mismatch ma con predizione)
found_df = df[df["pred_class"]!=""]
global_means = found_df[["iou","dice","precision","recall",
                        "specificity","balanced_acc","hausdorff_px"]].astype(float).mean()

# ────────── SCRITTURA FINALE CSV ──────────
with open("/content/Progetti_IA/SegmentazioneNoduliCerebrali/report/metrics_per_image.csv", "a", newline="") as f:
    w = csv.writer(f)

    # vuota
    w.writerow([])
    # Aggregati per classe
    w.writerow(["Aggregati per classe"])
    w.writerow(["gt_class","correct_preds","wrong_preds"])
    w.writerows(agg_rows)

    # vuota
    w.writerow([])
    # Global metrics
    w.writerow(["Global metrics (solo immagini con predizione)"])
    header = ["tumori_trovati","tumori_non_trovati"] + list(global_means.index)
    w.writerow(header)
    vals   = [n_found, n_not_found] + [f"{global_means[c]:.4f}" for c in global_means.index]
    w.writerow(vals)

print("✅ Tutto fatto! CSV finale: /content/Progetti_IA/SegmentazioneNoduliCerebrali/report/metrics_per_image.csv")


In [ ]:
### SCRIPT PER LETTURA REPORT CSV E GENERAZIONE REPORT METRICHE AGGREGATE CSV ###

import pandas as pd

IN_CSV  = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/report/metrics_per_image.csv"
OUT_CSV = "/content/Progetti_IA/SegmentazioneNoduliCerebrali/report/aggregate_metrics_per_class.csv"

# 1) Carica e filtra solo il tuo test set
df = pd.read_csv(IN_CSV)
df = df[df["image"].astype(str).str.startswith("brisc2025")].copy()

# 2) Normalizza 'match' a int
df["match"] = pd.to_numeric(df["match"], errors="coerce").fillna(0).astype(int)

# 3) Converti le metriche in float
metric_cols = ["iou","dice","precision","recall","specificity","balanced_acc","hausdorff_px"]
for col in metric_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
        .pipe(pd.to_numeric, errors="coerce")
        .fillna(0.0)
    )

# 4) Tieni solo le GT valide
valid_gt = ["Meningioma", "Glioma", "Pituitario"]
df = df[df["gt_class"].isin(valid_gt)].copy()

# --- DEBUG ---
print("Righe dopo filtro:", len(df))
print(df[["image","gt_class","pred_class","match"]].head())

# 5) Aggregazione
rows = []
for gt, grp in df.groupby("gt_class"):
    # conta predizioni corrette
    correct      = int((grp["match"] == 1).sum())
    # conta predizioni sbagliate: classi NON NaN e match==0
    wrong        = int(((grp["match"] == 0) & (~grp["pred_class"].isna())).sum())
    # conta non rilevati (pred_class NaN)
    not_detected = int(grp["pred_class"].isna().sum())

    # metrica media solo su correct==1
    corr_grp = grp[grp["match"] == 1]
    if len(corr_grp):
        agg = {c: corr_grp[c].mean() for c in metric_cols}
    else:
        agg = {c: 0.0 for c in metric_cols}

    rows.append({
        "gt_class":      gt,
        "correct_preds": correct,
        "wrong_preds":   wrong,
        "not_detected":  not_detected,
        **agg
    })

out_df = pd.DataFrame(rows, columns=[
    "gt_class","correct_preds","wrong_preds","not_detected", *metric_cols
])

# 6) Salva e stampa
out_df.to_csv(OUT_CSV, index=False, float_format="%.4f")
print(f"\n✅ Report aggregato per classe salvato in: {OUT_CSV}\n")
print(out_df.to_string(index=False, float_format="%.4f"))
